In [1]:
import pandas as pd
import numpy as np
import re
from gensim import matutils, models
import scipy.sparse
from nltk import word_tokenize, pos_tag

pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('offers.csv')
df = df.drop(['id'], axis=1)

In [3]:
# Remove Chinese offers
i = 0
while i < len(df)-1:
    x = df.iloc[i, :]
    if len(re.findall(r'[\u4e00-\u9fff]+', x['name'])) > 0:
        df = df.drop([i])
    i = i + 1
df = df.reset_index(drop=True)

In [4]:
df.dropna(subset=['desc'], inplace=True)
df = df.reset_index(drop=True)

In [5]:
# Detect Languages
from langdetect import detect

df['lang'] = ''

i = 0
while i <len(df)-1:
    x = df.iloc[i, :]['desc']
    df['lang'][i] = detect(x)
    i = i + 1
print('Done')

Done


In [6]:
df.head()

,name,link,company,function,details,desc,createdAt,updatedAt,lang
0,Payroll&Administration Specialist,https://emploi.thalesgroup.com/emploi/voivodie-de-mazovie/payroll-and-administration-specialist/17883/18564146,Thales,NaN,25/01/2021 /*/ Voïvodie de Mazovie,"Postuler Payroll&Administration Specialist ID de l'offre R0115527 Date de publication 25/01/2021 Lieu: Voïvodie de Mazovie, Poland Full time, Journalier Location: Warszawa, PolandThales people architect solutions that support 85 million mainline and suburban passenger journeys, worldwide, every day. Our Rail Signalling and Communication systems are used on metro lines across major cities, and 72,000 kms of route, 52,000 trains per day in 16 countries are controlled by our Traffic Management Systems. Together We deployed the first-ever nationwide ticketing system which processes over 50 million ticketing transactions in 100 cities daily.Thales is a global leader in the field of providing security systems for surface transport, airspace, defense operations and electronic transactions. In the field of rail transport, Thales is one of the leading suppliers of railway automation systems and a leader in the implementation of ETCS by providing automation of railway traffic control while maintaining the maximum safety of transport, passengers and goods. In Poland, Thales effectively supports the development of rail networks as a leader in the field of railway automation and develops new areas of business by offering technologically advanced solutions for the army, the aviation industry, transport, telecommunications and the public sector.• Koordynacja procesów zewnętrznej obsługi w zakresie administracji kadrowo-płacowej, w szczególności gromadzenie, weryfikacja i przekazywanie danych niezbędnych do wypłaty wynagrodzenia dla pracowników i osób na kontraktach cywilnoprawnych,• Koordynacja spraw kadrowo-płacowych w spółce we współpracy z firmą outsourcingową• Nadzór nad dokumentacją personalną• Przygotowywanie danych oraz nadzór nad prawidłowym i terminowym naliczaniem wynagrodzeń, benefitów i innych świadczeń dla pracowników, stażystów i współpracowników.• Ewidencja i aktualizacja danych pracowników w systemach informatycznych• Nadzór nad ewidencją i rozliczaniem czasu pracy, urlopów oraz zwolnień lekarskich.• Przygotowanie danych w procesie przeglądu wynagrodzeń i budżetowania• Przygotowanie danych w procesie zatrudnienia cudzoziemców w Polsce oraz oddelegowania pracowników polskich do innych oddziałów frmy poza granicami Polski• Aktualizacja Regulaminów Pracy, Wynagrodzeń i ZFŚS.• Przygotowywanie i opracowywanie określonych raportów kadrowo-płacowych• Wspieranie kadry zarządzającej w zakresie podejmowania kluczowych decyzji dotyczących m.in. zarządzania czasem pracy oraz innych kwestii związanych ze stosowaniem przepisów prawa pracy.• Prowadzenie procesów / projektów związanych z dopasowaniem zatrudnienia do potrzeb Spółki,• Wdrażanie i obsługa narzędzi wspomagających procesy HR,• Inne zadania związane z obszarem HR.At Thales we provide CAREERS and not only jobs. With Thales employing 80,000 employees in 68 countries our mobility policy enables thousands of employees each year to develop their careers at home and abroad, in their existing areas of expertise or by branching out into new fields. Together we believe that embracing flexibility is a smarter way of working. Great journeys start here, apply now! Postuler < Retourner aux résultats",2021-01-25 19:46:16.416+01,2021-01-25 19:46:16.416+01,pl
1,Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H,https://joinus.saint-gobain.com/fr/fra/spc/p/24470/7525/administrateur-ice-des-vente-plateforme-logistique-cdi-decoceram-fh,Saint Gobain,NaN,FRA01793 /*/ Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H France Auvergne Rhone Alpes CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01793 Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H Supply Chain France , Auvergne Rho

In [74]:
df_fr = df.loc[df['lang']== 'fr',:]
df_en = df.loc[df['lang']== 'en',:]

In [75]:
df_fr.head()

,name,link,company,function,details,desc,createdAt,updatedAt,lang
1,Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H,https://joinus.saint-gobain.com/fr/fra/spc/p/24470/7525/administrateur-ice-des-vente-plateforme-logistique-cdi-decoceram-fh,Saint Gobain,NaN,FRA01793 /*/ Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H France Auvergne Rhone Alpes CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01793 Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H Supply Chain France , Auvergne Rhone Alpes , SAINT PRIEST CDI Date de publication : 25/01/2021 Supply Chain France SAINT PRIEST CDI Date de publication : 25/01/2021 POSTULER Description du poste Au sein de la plateforme logistique de DECOCERAM St Priest : - vous êtes en charge de l'accueil téléphonique de notre réseau national de points de vente (60 en France), - vous répondrez aux équipes des points ventes sur la disponibilité des produits, les délais de livraisons, - vous préviendrez les points de vente des changements de planning de livraison, - vous serez le premier relai des réclamations des points de vente. PROFIL RECHERCHÉ Titulaire d'un bac + 2 en logistique, commerce ou similaire, vous êtes doté-e d'un excellent relationnel et avez des qualités pédagogiques reconnues qui font que avez le soucis de la satisfaction du client interne mais aussi du client final de nos points de vente. En véritable intermédiaire entre la plateforme logistique et nos points de ventes, vous maitrisez les outils de bureautique (messagerie Outlook, Word, Excel et Teams). Enfin, vous savez faire preuve d'esprit d'analyse, de synthèse et êtes force de proposition. D'autres raisons de nous rejoindre : · Un parcours d'intégration et une offre de formations personnalisées dans tous les domaines · Un suivi personnalisé tout au long de votre parcours dans le Groupe · Un accès aux avantages du Groupe Saint-Gobain (Plan Epargne Groupe, accords d'intéressement et participation) ainsi qu'à des remises commerciales sur nos produits dans la majorité de nos enseignes de distribution · Enfin, évoluer, changer de métier ou encore découvrir d'autres enseignes sera possible ! Qui sommes-nous ? Décocéram, enseigne de Saint-Gobain Distribution Bâtiment France, est un réseau d'enseignes spécialisées dans le carrelage et la décoration (plus de 50 points de vente). <iframe src=""https://pixtrack.meteojob.com/view/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> <iframe src=""https://pixtrack.meteojob.com/conv/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> CDI - Magasinier Cariste H/F - BOIS PANNEAUX - SAINT PRIEST (69) FRA01733 France Auvergne Rhone Alpes CDI 21/01/2021 CDI - Magasinier Cariste H/F - BOIS PANNEAUX - BOURG LES VALENCE (26) FRA01755 France Auvergne Rhone Alpes CDI 25/01/2021 Magasinier Cariste H/F CDI BOIS PANNEAUX CHAMBERY (73) FRA00783 France Auvergne Rhone Alpes CDI 08/12/2020",2021-01-25 19:46:15.507+01,2021-01-25 19:46:15.507+01,fr
3,Responsable Ressources Humaines - Siège H/F,https://joinus.saint-gobain.com/fr/fra/hr/p/46650/7530/responsable-ressources-humaines-siege-hf,Saint Gobain,NaN,FRA01782 /*/ Responsable Ressources Humaines - Siège H/F France Ile de France CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01782 Responsable Ressources Humaines - Siège H/F Ressources Humaines France , Ile de France , AUBERVILLIERS CDI Date de publication : 25/01/2021 Ressources Humaines France AUBERVILLIERS CDI Date de publication : 25/01/2021 POSTULER Description du poste Le Siège de Lapeyre regroupe l’ensemble des fonctions supports (env. 400 personnes) qui épaulent au quotidien nos magasins et nos usines. En véritable Business Partner vous accompagnez les Directeurs de Service sur le déploiement de la politique RH et la gestion des collaborateurs, dans un contexte de transformation et de changement d’a

In [76]:
df_fr['details'][0]

KeyError: 0

In [77]:
df_fr['details'] = df_fr['details'].apply(lambda x : str(x))
df_fr = df_fr.reset_index(drop=True)

contract_type = []
ct1 = []
ct2 = []
ct3 = []
x = 0
while x <= df_fr.shape[0]-1:
    target = df_fr['details'][x]
    
    c = re.findall('cdi|cdd|durée déterminée|durée indéterminée|regular full time', target.lower())
    if not c:
        ct1.append(c)
        #print(c)
    else:
        if c[0].upper() == 'DURÉE INDÉTERMINÉE':
            ct1.append('CDI')
            #print(c[0].upper())
        elif c[0].upper() == 'DURÉE DÉTERMINÉE':
            ct1.append('CDD')
            #print(c[0].upper())
        elif c[0].upper() == 'REGULAR FULL TIME':
            ct1.append('CDI')
            #print(c[0].upper())
        else:
            ct1.append(c[0].upper())
            #print(c[0].upper())

    targ = df_fr['desc'][x]
    
    r = re.findall('cdi|cdd|durée déterminée|durée indéterminée', targ.lower())
    if not r:
        ct2.append(r)
        #print(r)
    else:
        if r[0].upper() == 'DURÉE INDÉTERMINÉE':
            ct2.append('CDI')
            #print(r[0].upper())
        elif r[0].upper() == 'DURÉE DÉTERMINÉE':
            ct2.append('CDD')
            #print(r[0].upper())
        else:
            ct2.append(r[0].upper())
            #print(r[0].upper())
            
    search_field = df_fr['name'][x]
    
    res = re.findall('cdi|cdd|durée déterminée|durée indéterminée', search_field.lower())
    if not res:
        ct3.append(res)
        #print(r)
    else:
        if res[0].upper() == 'DURÉE INDÉTERMINÉE':
            ct3.append('CDI')
            #print(r[0].upper())
        elif res[0].upper() == 'DURÉE DÉTERMINÉE':
            ct3.append('CDD')
            #print(r[0].upper())
        else:
            ct3.append(res[0].upper())
            #print(r[0].upper())
    x = x + 1

x = 0
while x <= df_fr.shape[0]-1:
    if not ct1[x]:
        if not ct2[x]:
            if not ct3[x]:
                contract_type.append('Unspecified')
            else:
                contract_type.append(ct3[x])
        else:
            contract_type.append(ct2[x])
    else:
        contract_type.append(ct1[x])
    x = x + 1
    
df_fr['contract_type'] = contract_type

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
df_fr.head(10)

name  \
0                                                            Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H   
1                                                                                     Responsable Ressources Humaines - Siège H/F   
2                             CHEF DE PRODUIT SENIOR PRODUITS FINIS - CDD (H/F) CHEF DE PRODUIT SENIOR PRODUITS FINIS - CDD (H/F)   
3                                                                                CDI-Responsable Industriel d'Offre Nucléaire H/F   
4                                                               CDI - Responsable projet / Gestionnaire de Configuration IT (H/F)   
5                                                            CDI - Ingénieur cybersécurité confirmé en développement embarqué H/F   
6  GESTIONNAIRE PAIE ET ADMINISTRATION DU PERSONNEL H/F Published on 01/25/2021 Temara, Morocco Regular Full time Human Resources   
7                                                                                                            CDI - Frigoriste H/F   
8                                                                                                     Project Planner / Scheduler   
9                                                                                            MISE A JOUR DE DONNEES / POSTE 2 H/F   

                                                                                                                          link  \
0  https://joinus.saint-gobain.com/fr/fra/spc/p/24470/7525/administrateur-ice-des-vente-plateforme-logistique-cdi-decoceram-fh   
1                              https://joinus.saint-gobain.com/fr/fra/hr/p/46650/7530/responsable-ressources-humaines-siege-hf   
2                  https://jobs.richemont.com/Chloe/job/paris-CHEF-DE-PRODUIT-SENIOR-PRODUITS-FINIS-CDD-%28HF%29-75/645603201/   
3                                                                  https://www.naval-group.com/fr/talents/offer/P2019-003350-1   
4                                                                  https://www.naval-group.com/fr/talents/offer/P2020-000787-1   
5                                                                  https://www.naval-group.com/fr/talents/offer/P2018-003138-1   
6                           https://www.safran-group.com/job/temara/gestionnaire-paie-et-administration-du-personnel-h-f/86736   
7                                                                  https://www.naval-group.com/fr/talents/offer/P2017-015763-1   
8                                       https://emploi.thalesgroup.com/emploi/toronto/project-planner-scheduler/17883/18562301   
9                   https://carriere.dassault-aviation.com/offre-de-emploi/emploi-mise-a-jour-de-donnees-poste-2-h-f_6779.aspx   

             company                     function  \
0       Saint Gobain                          NaN   
1       Saint Gobain                          NaN   
2              Chloé                    Marketing   
3        Naval Group  Commerce, marketing , Offre   
4        Naval Group                   Programmes   
5        Naval Group         Informatique produit   
6             Safran                          NaN   
7        Naval Group                    Mécanique   
8             Thales                          NaN   
9  Dassault Aviation                          NaN   

                                                                                                                        details  \
0  FRA01793 /*/ Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H France Auvergne Rhone Alpes CDI 25/01/2021   
1                                  FRA01782 /*/ Responsable Ressources Humaines - Siège H/F France Ile de France CDI 25/01/2021   
2                                                                                                                 paris, 75, FR   
3                                                         Commerce, marketing , Offre /*/ Durée indéterminée France , Cher

In [79]:
df_fr['contract_type'].value_counts()

CDI            1889
Unspecified     271
CDD             223
Name: contract_type, dtype: int64

In [80]:
pays = []
region = []
dpt = []
ville = []
x = 0
while x <= df_fr.shape[0]-1:
    target = df_fr['details'][x]
    
    c = re.findall('france|canada|tunisia', target.lower())
    if not c:
        pays.append(c)
    else:
        pays.append(c[0].title())
        
    r = re.findall("wallonie|île-de-france|normandie|bretagne|ile de france|hauts de france|centre val de loire|provence/alps|hauts-de-france|quebec|québec|auvergne-rhône-alpes|bourgogne franche comte|pays de la loire|provence-alpes-côte d'azur|grand est|nouvelle-aquitaine|occitanie", target.lower())
    if not r:
        region.append(r)
    else:
        region.append(r[0].title())
                
    t = re.findall('haute normandie|gironde|gard|manche|eure|hauts-de-seine|seine et marne|savoie|haute savoie|isère|rhône|loire atlantique|saône et loire|bouches du rhône|hérault', target.lower())
    if not t:
        dpt.append(t)
    else:
        dpt.append(t[0].title())
        
    
    v = re.findall('lille|marseille|saint nazaire|saint-nazaire|portet sur garonne|st brieuc|wattrelos|compiègne|bouguenais|dampierre en burly|dampierre-en-burly|angers|albertville|massy|libourne|pau|saumur|saint-denis|grenoble|dhari|rungis|gardanne|lens|ollioules|bagneux|nevers|mérignac|montréal|montreal|saint-ouen|saint ouen|epinal|soliman|carquefou|bordes|rouen|pontarlier|velizy|gennevilliers|montpellier|eyguières|saint lô|tulle|le havre|lannion|niort|saint brieuc|lorient|arras|auxerre|colombelles|pont-audemer|feyzin|issy-les-moulineaux|palaiseau|cholet|limours|puilboreau|mantes la jolie|alençon|st maximin|pierrelatte|bourg en bresse|metz|laval|granville|avranches|toulon|beauvais|gignac|langueux|beaucouzé|la chapelle saint ursin|roanne|st sebastien|saint denis|abbeville|domerat|la défense|la defense|nanterre|lisieux|bagnolet|bourges|haguenau|saran|nantes|brive-la-gaillarde|menton|charleville-mézières|saint-dizier|versailles|toulouse|thionville|chauny|le caire|valenciennes|pantin|avignon|rians|aubervilliers|caen|rennes|courbevoie|annecy|la courneuve|le plessis-robinson|orléans|ivry sur seine|brest|eragny|cherbourg|paris|lyon|vélizy-villacoublay|élancourt', target.lower())
    if not v:
        ville.append(v)
    else:
        ville.append(v[0].title())

        
        
    #if not ville[x]:
        #print(df_fr['details'][x])
        #if not region[x]:
            #if not dpt[x]:
                #if not pays[x]:
                    #print(df_fr['details'][x])
                #else:
                    #print(pays[x])
            #else:
                #print(dpt[x])
        #else:
            #print(region[x])
    #else:
        #print(ville[x])
        
        
    print(f'Pays : {pays[x]}, Region : {region[x]}, Departement : {dpt[x]}, Ville : {ville[x]}')
    x = x + 1
    
    
df_fr['country'] = pays
df_fr['region'] = region
df_fr['departement'] = dpt
df_fr['ville'] = ville

Pays : France, Region : [], Departement : [], Ville : []
Pays : France, Region : Ile De France, Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Paris
Pays : France, Region : [], Departement : [], Ville : Cherbourg
Pays : France, Region : [], Departement : [], Ville : Cherbourg
Pays : France, Region : [], Departement : [], Ville : Ollioules
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : Cherbourg
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : France, Region : Occitanie, Departement : [], Ville : []
Pays : [], Region : [], Departement : Rhône, Ville : []
Pays : [], Region : [], Departement : Hauts-De-Seine, Ville : []
Pays : France, Region : Île-De-France, Departement : [],

Pays : France, Region : Provence/Alps, Departement : [], Ville : []
Pays : France, Region : Hauts De France, Departement : [], Ville : []
Pays : France, Region : Ile De France, Departement : [], Ville : []
Pays : France, Region : Bourgogne Franche Comte, Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Versailles
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Arras
Pays : [], Region : [], Departement : [], Ville : Portet Sur Garonne
Pays : France, Region : [], Departement : [], Ville : Cherbourg
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : Hauts-De-Seine, Ville : []
Pays : France, Region : [], Departement : [], Ville : Toulon
Pays : [], Region : [], Departement : [], Ville : Ivry Sur Seine
Pays : [], Region : [], Departement : Hauts-De-Seine, Ville : [

Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Versailles
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : Canada, Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Roanne
Pays : [], Region : Occitanie, Departement : [], Ville : Toulouse
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Lille
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Auxerre
Pays : [], Region : [], Departement : [], Vill

Pays : France, Region : Occitanie, Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : Hauts-De-Seine, Ville : []
Pays : France, Region : [], Departement : [], Ville : Valenciennes
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : Occitanie, Departement : [], Ville : Toulouse
Pays : France, Region : [], Departement : [], Ville : Valenciennes
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : Hauts-De-Seine, Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : France, Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : []
Pays : [], Region : [], Departement : [], Ville : Paris
Pays : [], Region : [], Departement : Hauts-De-Seine, Vil

In [81]:
df_fr.head()

,name,link,company,function,details,desc,createdAt,updatedAt,lang,contract_type,country,region,departement,ville
0,Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H,https://joinus.saint-gobain.com/fr/fra/spc/p/24470/7525/administrateur-ice-des-vente-plateforme-logistique-cdi-decoceram-fh,Saint Gobain,NaN,FRA01793 /*/ Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H France Auvergne Rhone Alpes CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01793 Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H Supply Chain France , Auvergne Rhone Alpes , SAINT PRIEST CDI Date de publication : 25/01/2021 Supply Chain France SAINT PRIEST CDI Date de publication : 25/01/2021 POSTULER Description du poste Au sein de la plateforme logistique de DECOCERAM St Priest : - vous êtes en charge de l'accueil téléphonique de notre réseau national de points de vente (60 en France), - vous répondrez aux équipes des points ventes sur la disponibilité des produits, les délais de livraisons, - vous préviendrez les points de vente des changements de planning de livraison, - vous serez le premier relai des réclamations des points de vente. PROFIL RECHERCHÉ Titulaire d'un bac + 2 en logistique, commerce ou similaire, vous êtes doté-e d'un excellent relationnel et avez des qualités pédagogiques reconnues qui font que avez le soucis de la satisfaction du client interne mais aussi du client final de nos points de vente. En véritable intermédiaire entre la plateforme logistique et nos points de ventes, vous maitrisez les outils de bureautique (messagerie Outlook, Word, Excel et Teams). Enfin, vous savez faire preuve d'esprit d'analyse, de synthèse et êtes force de proposition. D'autres raisons de nous rejoindre : · Un parcours d'intégration et une offre de formations personnalisées dans tous les domaines · Un suivi personnalisé tout au long de votre parcours dans le Groupe · Un accès aux avantages du Groupe Saint-Gobain (Plan Epargne Groupe, accords d'intéressement et participation) ainsi qu'à des remises commerciales sur nos produits dans la majorité de nos enseignes de distribution · Enfin, évoluer, changer de métier ou encore découvrir d'autres enseignes sera possible ! Qui sommes-nous ? Décocéram, enseigne de Saint-Gobain Distribution Bâtiment France, est un réseau d'enseignes spécialisées dans le carrelage et la décoration (plus de 50 points de vente). <iframe src=""https://pixtrack.meteojob.com/view/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> <iframe src=""https://pixtrack.meteojob.com/conv/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> CDI - Magasinier Cariste H/F - BOIS PANNEAUX - SAINT PRIEST (69) FRA01733 France Auvergne Rhone Alpes CDI 21/01/2021 CDI - Magasinier Cariste H/F - BOIS PANNEAUX - BOURG LES VALENCE (26) FRA01755 France Auvergne Rhone Alpes CDI 25/01/2021 Magasinier Cariste H/F CDI BOIS PANNEAUX CHAMBERY (73) FRA00783 France Auvergne Rhone Alpes CDI 08/12/2020",2021-01-25 19:46:15.507+01,2021-01-25 19:46:15.507+01,fr,CDI,France,[],[],[]
1,Responsable Ressources Humaines - Siège H/F,https://joinus.saint-gobain.com/fr/fra/hr/p/46650/7530/responsable-ressources-humaines-siege-hf,Saint Gobain,NaN,FRA01782 /*/ Responsable Ressources Humaines - Siège H/F France Ile de France CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01782 Responsable Ressources Humaines - Siège H/F Ressources Humaines France , Ile de France , AUBERVILLIERS CDI Date de publication : 25/01/2021 Ressources Humaines France AUBERVILLIERS CDI Date de publication : 25/01/2021 POSTULER Description du poste Le Siège de Lapeyre regroupe l’ensemble des fonctions supports (env. 400 personnes) qui épaulent au quotidien nos magasins et nos usines. En véritable Business Partner vous accompagnez les Directeurs de Service sur le déploiement de la politique RH et la gestion des coll

In [82]:
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('H/F', ''))
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('F/H', ''))
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('CDI', ''))
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('CDD', ''))
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('- ', ''))
df_fr['name'] = df_fr['name'].apply(lambda x : x.replace('()', ''))

In [83]:
x = df_fr['name'].value_counts()
print(x)

CONCEPTEUR VENDEUR CUISINE                                                   76
VENDEUR                                                                      53
CONSEILLER POLE SERVICES                                                     27
Assistant(e) d'équipe                                                        19
RESPONSABLE DES VENTES CUISINE                                               14
                                                                             ..
Responsable Performances Systèmes-                                            1
Contrôleur(se) de gestion industriel et Projet en /Détachement de 12 mois     1
Chef de projet MOE 3DEXPERIENCE                                               1
Vendeur Conseil   HAGUENAU                                                    1
Ingénieur calculs mécaniques expérimenté sur les générateurs de vapeur        1
Name: name, Length: 1655, dtype: int64


In [84]:
import string

def clean_text(text):
    text = text.lower()

    #text = re.sub("de|le|les|vous|s|et|la|en|un|du|à|pour|une|dans|nous|h|f", '', text)
    text = re.sub("l'", '', text)
    text = re.sub("d'", '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('[0-9]', '', text)
    text = re.sub('•', '', text)
    text = re.sub('', '', text)
    text = re.sub("\…", '', text)
    text = re.sub('   ', ' ', text)
    text = re.sub('  ', ' ', text)
    return text

def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

french_stop_words = ["a", "à", "â", "abord", "afin", "ah", "ai", "aie", "ainsi", "allaient", "allo", "allô", "allons", "alors", "après", "assez", "attendu", "au", "aucun", "aucune", "aucuns", "aujourd", "aujourd'hui", "auquel", "aura", "auront", "aussi", "autre", "autres", "aux", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avoir", "ayant", "b", "bah", "beaucoup", "bien", "bigre", "bon", "boum", "bravo", "brrr", "c", "ça", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "cent", "cependant", "certain", "certaine", "certaines", "certains", "certes", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chacun", "chaque", "cher", "chère", "chères", "chers", "chez", "chiche", "chut", "ci", "cinq", "cinquantaine", "cinquante", "cinquantième", "cinquième", "clac", "clic", "combien", "comme", "comment", "compris", "concernant", "contre", "couic", "crac", "d", "da", "dans", "de", "debout", "début", "dedans", "dehors", "delà", "depuis", "derrière", "des", "dès", "désormais", "desquelles", "desquels", "dessous", "dessus", "deux", "deuxième", "deuxièmement", "devant", "devers", "devra", "devrait", "différent", "différente", "différentes", "différents", "dire", "divers", "diverse", "diverses", "dix", "dix-huit", "dixième", "dix-neuf", "dix-sept", "doit", "doivent", "donc", "dont", "dos", "douze", "douzième", "dring", "droite", "du", "duquel", "durant", "e", "effet", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "entre", "envers", "environ", "es", "ès", "essai", "est", "et", "étaient", "étais", "était", "etant", "étant", "état", "etc", "été", "étions", "etre", "être", "eu", "euh", "eux", "eux-mêmes", "excepté", "f", "façon", "fais", "faisaient", "faisant", "fait", "faites", "feront", "fi", "flac", "floc", "fois", "font", "force", "g", "gens", "h", "ha", "haut", "hé", "hein", "hélas", "hem", "hep", "hi", "ho", "holà", "hop", "hormis", "hors", "hou", "houp", "hue", "hui", "huit", "huitième", "hum", "hurrah", "i", "ici", "il", "ils", "importe", "j", "je", "jusqu", "jusque", "juste", "k", "l", "la", "là", "laquelle", "las", "le", "lequel", "les", "lès", "lesquelles", "lesquels", "leur", "leurs", "longtemps", "lorsque", "lui", "lui-même", "m", "ma", "maint", "maintenant", "mais", "malgré", "me", "même", "mêmes", "merci", "mes", "mien", "mienne", "miennes", "miens", "mille", "mince", "mine", "moi", "moi-même", "moins", "mon", "mot", "moyennant", "n", "na", "ne", "néanmoins", "neuf", "neuvième", "ni", "nombreuses", "nombreux", "nommés", "non", "nos", "notre", "nôtre", "nôtres", "nous", "nous-mêmes", "nouveaux", "nul", "o", "ô", "o|", "oh", "ohé", "olé", "ollé", "on", "ont", "onze", "onzième", "ore", "ou", "où", "ouf", "ouias", "oust", "ouste", "outre", "p", "paf", "pan", "par", "parce", "parmi", "parole", "partant", "particulier", "particulière", "particulièrement", "pas", "passé", "pendant", "personne", "personnes", "peu", "peut", "peuvent", "peux", "pff", "pfft", "pfut", "pièce", "pif", "plein", "plouf", "plupart", "plus", "plusieurs", "plutôt", "pouah", "pour", "pourquoi", "premier", "première", "premièrement", "près", "proche", "psitt", "puisque", "q", "qu", "quand", "quant", "quanta", "quant-à-soi", "quarante", "quatorze", "quatre", "quatre-vingt", "quatrième", "quatrièmement", "que", "quel", "quelconque", "quelle", "quelles", "quelque", "quelques", "quelqu'un", "quels", "qui", "quiconque", "quinze", "quoi", "quoique", "r", "revoici", "revoilà", "rien", "s", "sa", "sacrebleu", "sans", "sapristi", "sauf", "se", "seize", "selon", "sept", "septième", "sera", "seront", "ses", "seulement", "si", "sien", "sienne", "siennes", "siens", "sinon", "six", "sixième", "soi", "soi-même", "soit", "soixante", "son", "sont", "sous", "soyez", "stop", "suis", "suivant", "sujet", "sur", "surtout", "t", "ta", "tac", "tandis", "tant", "te", "té", "tel", "telle", "tellement", "telles", "tels", "tenant", "tes", "tic", "tien", "tienne", "tiennes", "tiens", "toc", "toi", "toi-même", "ton", "touchant", "toujours", "tous", "tout", "toute", "toutes", "treize", "trente", "très", "trois", "troisième", "troisièmement", "trop", "tsoin", "tsouin", "tu", "u", "un", "une", "unes", "uns", "v", "va", "vais", "valeur", "vas", "vé", "vers", "via", "vif", "vifs", "vingt", "vivat", "vive", "vives", "vlan", "voici", "voie", "voient", "voilà", "vont", "vos", "votre", "vôtre", "vôtres", "vous", "vous-mêmes", "vu", "w", "x", "y", "z", "zut","a", "abord", "absolument", "afin", "ah", "ai", "aie", "aient", "aies", "ailleurs", "ainsi", "ait", "allaient", "allo", "allons", "allô", "alors", "anterieur", "anterieure", "anterieures", "apres", "après", "as", "assez", "attendu", "au", "aucun", "aucune", "aucuns", "aujourd", "aujourd'hui", "aupres", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autant", "autre", "autrefois", "autrement", "autres", "autrui", "aux", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avoir", "avons", "ayant", "ayez", "ayons", "b", "bah", "bas", "basee", "bat", "beau", "beaucoup", "bien", "bigre", "bon", "boum", "bravo", "brrr", "c", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cependant", "certain", "certaine", "certaines", "certains", "certes", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chacun", "chacune", "chaque", "cher", "chers", "chez", "chiche", "chut", "chère", "chères", "ci", "cinq", "cinquantaine", "cinquante", "cinquantième", "cinquième", "clac", "clic", "combien", "comme", "comment", "comparable", "comparables", "compris", "concernant", "contre", "couic", "crac", "d", "da", "dans", "de", "debout", "dedans", "dehors", "deja", "delà", "depuis", "dernier", "derniere", "derriere", "derrière", "des", "desormais", "desquelles", "desquels", "dessous", "dessus", "deux", "deuxième", "deuxièmement", "devant", "devers", "devra", "devrait", "different", "differentes", "differents", "différent", "différente", "différentes", "différents", "dire", "directe", "directement", "dit", "dite", "dits", "divers", "diverse", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dixième", "doit", "doivent", "donc", "dont", "dos", "douze", "douzième", "dring", "droite", "du", "duquel", "durant", "dès", "début", "désormais", "e", "effet", "egale", "egalement", "egales", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "enfin", "entre", "envers", "environ", "es", "essai", "est", "et", "etant", "etc", "etre", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eux-mêmes", "exactement", "excepté", "extenso", "exterieur", "eûmes", "eût", "eûtes", "f", "fais", "faisaient", "faisant", "fait", "faites", "façon", "feront", "fi", "flac", "floc", "fois", "font", "force", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gens", "h", "ha", "haut", "hein", "hem", "hep", "hi", "ho", "holà", "hop", "hormis", "hors", "hou", "houp", "hue", "hui", "huit", "huitième", "hum", "hurrah", "hé", "hélas", "i", "ici", "il", "ils", "importe", "j", "je", "jusqu", "jusque", "juste", "k", "l", "la", "laisser", "laquelle", "las", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "longtemps", "lors", "lorsque", "lui", "lui-meme", "lui-même", "là", "lès", "m", "ma", "maint", "maintenant", "mais", "malgre", "malgré", "maximale", "me", "meme", "memes", "merci", "mes", "mien", "mienne", "miennes", "miens", "mille", "mince", "mine", "minimale", "moi", "moi-meme", "moi-même", "moindres", "moins", "mon", "mot", "moyennant", "multiple", "multiples", "même", "mêmes", "n", "na", "naturel", "naturelle", "naturelles", "ne", "neanmoins", "necessaire", "necessairement", "neuf", "neuvième", "ni", "nombreuses", "nombreux", "nommés", "non", "nos", "notamment", "notre", "nous", "nous-mêmes", "nouveau", "nouveaux", "nul", "néanmoins", "nôtre", "nôtres", "o", "oh", "ohé", "ollé", "olé", "on", "ont", "onze", "onzième", "ore", "ou", "ouf", "ouias", "oust", "ouste", "outre", "ouvert", "ouverte", "ouverts", "o|", "où", "p", "paf", "pan", "par", "parce", "parfois", "parle", "parlent", "parler", "parmi", "parole", "parseme", "partant", "particulier", "particulière", "particulièrement", "pas", "passé", "pendant", "pense", "permet", "personne", "personnes", "peu", "peut", "peuvent", "peux", "pff", "pfft", "pfut", "pif", "pire", "pièce", "plein", "plouf", "plupart", "plus", "plusieurs", "plutôt", "possessif", "possessifs", "possible", "possibles", "pouah", "pour", "pourquoi", "pourrais", "pourrait", "pouvait", "prealable", "precisement", "premier", "première", "premièrement", "pres", "probable", "probante", "procedant", "proche", "près", "psitt", "pu", "puis", "puisque", "pur", "pure", "q", "qu", "quand", "quant", "quant-à-soi", "quanta", "quarante", "quatorze", "quatre", "quatre-vingt", "quatrième", "quatrièmement", "que", "quel", "quelconque", "quelle", "quelles", "quelqu'un", "quelque", "quelques", "quels", "qui", "quiconque", "quinze", "quoi", "quoique", "r", "rare", "rarement", "rares", "relative", "relativement", "remarquable", "rend", "rendre", "restant", "reste", "restent", "restrictif", "retour", "revoici", "revoilà", "rien", "s", "sa", "sacrebleu", "sait", "sans", "sapristi", "sauf", "se", "sein", "seize", "selon", "semblable", "semblaient", "semble", "semblent", "sent", "sept", "septième", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "seul", "seule", "seulement", "si", "sien", "sienne", "siennes", "siens", "sinon", "six", "sixième", "soi", "soi-même", "soient", "sois", "soit", "soixante", "sommes", "son", "sont", "sous", "souvent", "soyez", "soyons", "specifique", "specifiques", "speculatif", "stop", "strictement", "subtiles", "suffisant", "suffisante", "suffit", "suis", "suit", "suivant", "suivante", "suivantes", "suivants", "suivre", "sujet", "superpose", "sur", "surtout", "t", "ta", "tac", "tandis", "tant", "tardive", "te", "tel", "telle", "tellement", "telles", "tels", "tenant", "tend", "tenir", "tente", "tes", "tic", "tien", "tienne", "tiennes", "tiens", "toc", "toi", "toi-même", "ton", "touchant", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "tres", "trois", "troisième", "troisièmement", "trop", "très", "tsoin", "tsouin", "tu", "té", "u", "un", "une", "unes", "uniformement", "unique", "uniques", "uns", "v", "va", "vais", "valeur", "vas", "vers", "via", "vif", "vifs", "vingt", "vivat", "vive", "vives", "vlan", "voici", "voie", "voient", "voilà", "voire", "vont", "vos", "votre", "vous", "vous-mêmes", "vu", "vé", "vôtre", "vôtres", "w", "x", "y", "z", "zut", "à", "â", "ça", "ès", "étaient", "étais", "était", "étant", "état", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"]

def removeStopwords(wordlist, stopwords):
    return [w for w in wordlist if w not in stopwords]

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

def get_cleantext(x, stopwords):
    wordlist = removeStopwords(x.split(), french_stop_words)
    frqList = wordListToFreqDict(wordlist)
    sortFrqList = sortFreqDict(frqList)
    return sortFrqList

df_fr['clean_desc'] = df_fr['desc'].apply(lambda x : clean_text(x))
df_fr['clean_desc'] = df_fr['clean_desc'].apply(lambda x : get_cleantext(x, french_stop_words))

In [85]:
df_fr.head()

,name,link,company,function,details,desc,createdAt,updatedAt,lang,contract_type,country,region,departement,ville,clean_desc
0,Administrateur-ice des Vente Plateforme Logistique DECOCERAM,https://joinus.saint-gobain.com/fr/fra/spc/p/24470/7525/administrateur-ice-des-vente-plateforme-logistique-cdi-decoceram-fh,Saint Gobain,NaN,FRA01793 /*/ Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H France Auvergne Rhone Alpes CDI 25/01/2021,"Retour à la liste des offres Référence : FRA01793 Administrateur-ice des Vente Plateforme Logistique CDI DECOCERAM F/H Supply Chain France , Auvergne Rhone Alpes , SAINT PRIEST CDI Date de publication : 25/01/2021 Supply Chain France SAINT PRIEST CDI Date de publication : 25/01/2021 POSTULER Description du poste Au sein de la plateforme logistique de DECOCERAM St Priest : - vous êtes en charge de l'accueil téléphonique de notre réseau national de points de vente (60 en France), - vous répondrez aux équipes des points ventes sur la disponibilité des produits, les délais de livraisons, - vous préviendrez les points de vente des changements de planning de livraison, - vous serez le premier relai des réclamations des points de vente. PROFIL RECHERCHÉ Titulaire d'un bac + 2 en logistique, commerce ou similaire, vous êtes doté-e d'un excellent relationnel et avez des qualités pédagogiques reconnues qui font que avez le soucis de la satisfaction du client interne mais aussi du client final de nos points de vente. En véritable intermédiaire entre la plateforme logistique et nos points de ventes, vous maitrisez les outils de bureautique (messagerie Outlook, Word, Excel et Teams). Enfin, vous savez faire preuve d'esprit d'analyse, de synthèse et êtes force de proposition. D'autres raisons de nous rejoindre : · Un parcours d'intégration et une offre de formations personnalisées dans tous les domaines · Un suivi personnalisé tout au long de votre parcours dans le Groupe · Un accès aux avantages du Groupe Saint-Gobain (Plan Epargne Groupe, accords d'intéressement et participation) ainsi qu'à des remises commerciales sur nos produits dans la majorité de nos enseignes de distribution · Enfin, évoluer, changer de métier ou encore découvrir d'autres enseignes sera possible ! Qui sommes-nous ? Décocéram, enseigne de Saint-Gobain Distribution Bâtiment France, est un réseau d'enseignes spécialisées dans le carrelage et la décoration (plus de 50 points de vente). <iframe src=""https://pixtrack.meteojob.com/view/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> <iframe src=""https://pixtrack.meteojob.com/conv/n0DLrqEGXbkQMJKEpaxcOA?offerRef=REF"" height=""0"" width=""0"" style=""display:none;visibility:hidden""></iframe> CDI - Magasinier Cariste H/F - BOIS PANNEAUX - SAINT PRIEST (69) FRA01733 France Auvergne Rhone Alpes CDI 21/01/2021 CDI - Magasinier Cariste H/F - BOIS PANNEAUX - BOURG LES VALENCE (26) FRA01755 France Auvergne Rhone Alpes CDI 25/01/2021 Magasinier Cariste H/F CDI BOIS PANNEAUX CHAMBERY (73) FRA00783 France Auvergne Rhone Alpes CDI 08/12/2020",2021-01-25 19:46:15.507+01,2021-01-25 19:46:15.507+01,fr,CDI,France,[],[],[],"[(9, cdi), (7, points), (7, france), (6, vente), (5, saint), (4, ·), (4, rhone), (4, priest), (4, logistique), (4, iframe), (4, fra), (4, auvergne), (4, alpes), (3, plateforme), (3, panneaux), (3, magasinier), (3, groupe), (3, enseignes), (3, cariste), (3, bois), (2, width), (2, visibility), (2, ventes), (2, supply), (2, style), (2, src), (2, réseau), (2, ref), (2, publication), (2, produits), (2, pixtrack), (2, parcours), (2, offerref), (2, none), (2, ndlrqegxbkqmjkepaxcoa), (2, meteojob), (2, https), (2, hidden), (2, height), (2, gobain), (2, distribution), (2, display), (2, decoceram), (2, date), (2, com), (2, client), (2, chain), (1, évoluer), (1, équipes), (1, word), (1, véritable), (1, view), (1, valence), (1, téléphonique), (1, titulaire), (1, teams), (1, synthèse), (1, suivi), (1, st), (1, spécialis

In [86]:
df_fr['function'].value_counts()
df_fr['function']= df_fr['function'].replace(["Système d'information d'entreprise", 'Sourcing', 'Achat', 'Achats', 'IT', 'Informatique produit', 'Supply Chain & Planning', 'Ressources humaines', 'Human Resources', 'Ressources Humaines', 'Logistique, Supply chain', 'Supply Chain',
                                              'Supply Chain & Logistics', 'Engineering',
                                              'Etudes et ingénierie'],
                                             ['IT', 'Achats', 'Achats', 'Achats', 'IT', 'IT', 'Supply Chain et Logistique', 'Ressources Humaines', 'Ressources Humaines', 'Ressources Humaines', 'Supply Chain et Logistique',
                                              'Supply Chain et Logistique', 'Supply Chain et Logistique', 
                                              'Ingéniérie', 'Ingéniérie'])

In [87]:
df_fr['function'].value_counts()

Ingéniérie                             104
Supply Chain et Logistique              24
IT                                      22
Industrial                              20
Project & Bid Management                19
Programmes                              18
Ressources Humaines                     14
Manufacturing                           12
Sales                                   11
Electricité                             10
ENGIE Solutions                          8
Achats                                   7
Soutien général                          7
Mécanique                                7
Engie Solutions                          6
Services industriels                     6
Finance                                  6
Pilotage Industriel                      6
Electronique                             5
Méthode et Industrialisation             5
Contrôle, mesure, essais                 5
Nucléaire                                4
Marketing                                4
Installatio

In [33]:
df_fr.to_excel("offers_unlabeled.xlsx")

In [98]:
len(df_fr)

2383

In [99]:
#df_fr['name'] = df_fr['name'].replace(r"^-", '')
df_fr['name'] = df_fr['name'].apply(lambda x : x.lstrip('-'))
df_fr['name'] = df_fr['name'].apply(lambda x : x.lstrip('\w'))
#df_fr['name'] = df_fr['name'].apply(lambda x : x.lstrip(' '))

In [100]:
df_fr['name'].value_counts()

CONCEPTEUR VENDEUR CUISINE                 76
VENDEUR                                    53
CONSEILLER POLE SERVICES                   27
Assistant(e) d'équipe                      19
RESPONSABLE DES VENTES CUISINE             14
                                           ..
Assistant (e) commercial (e)                1
Développeur JAVA junior                     1
Infirmier(e) du travail                     1
Responsable Performances Systèmes-          1
Responsable Commercial  Agence de BERCY     1
Name: name, Length: 1649, dtype: int64

In [ ]:
test_offer = df.iloc[2,:]
test_offer['desc']

In [ ]:
# text cleaning


test_offer_text = clean_text(test_offer['desc'])
test_offer_text

In [ ]:
corpus_fr = df_fr[['name', 'company', 'desc']]
corpus_fr['name'] = corpus_fr['name'] + ' - ' + corpus_fr['company']
corpus_fr = corpus_fr.drop(columns=('company'))
corpus_fr

In [ ]:
corpus_fr['desc'] = corpus_fr['desc'].apply(lambda x : clean_text(x)) 

In [ ]:
french_stop_words = ["a", "à", "â", "abord", "afin", "ah", "ai", "aie", "ainsi", "allaient", "allo", "allô", "allons", "alors", "après", "assez", "attendu", "au", "aucun", "aucune", "aucuns", "aujourd", "aujourd'hui", "auquel", "aura", "auront", "aussi", "autre", "autres", "aux", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avoir", "ayant", "b", "bah", "beaucoup", "bien", "bigre", "bon", "boum", "bravo", "brrr", "c", "ça", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "cent", "cependant", "certain", "certaine", "certaines", "certains", "certes", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chacun", "chaque", "cher", "chère", "chères", "chers", "chez", "chiche", "chut", "ci", "cinq", "cinquantaine", "cinquante", "cinquantième", "cinquième", "clac", "clic", "combien", "comme", "comment", "compris", "concernant", "contre", "couic", "crac", "d", "da", "dans", "de", "debout", "début", "dedans", "dehors", "delà", "depuis", "derrière", "des", "dès", "désormais", "desquelles", "desquels", "dessous", "dessus", "deux", "deuxième", "deuxièmement", "devant", "devers", "devra", "devrait", "différent", "différente", "différentes", "différents", "dire", "divers", "diverse", "diverses", "dix", "dix-huit", "dixième", "dix-neuf", "dix-sept", "doit", "doivent", "donc", "dont", "dos", "douze", "douzième", "dring", "droite", "du", "duquel", "durant", "e", "effet", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "entre", "envers", "environ", "es", "ès", "essai", "est", "et", "étaient", "étais", "était", "etant", "étant", "état", "etc", "été", "étions", "etre", "être", "eu", "euh", "eux", "eux-mêmes", "excepté", "f", "façon", "fais", "faisaient", "faisant", "fait", "faites", "feront", "fi", "flac", "floc", "fois", "font", "force", "g", "gens", "h", "ha", "haut", "hé", "hein", "hélas", "hem", "hep", "hi", "ho", "holà", "hop", "hormis", "hors", "hou", "houp", "hue", "hui", "huit", "huitième", "hum", "hurrah", "i", "ici", "il", "ils", "importe", "j", "je", "jusqu", "jusque", "juste", "k", "l", "la", "là", "laquelle", "las", "le", "lequel", "les", "lès", "lesquelles", "lesquels", "leur", "leurs", "longtemps", "lorsque", "lui", "lui-même", "m", "ma", "maint", "maintenant", "mais", "malgré", "me", "même", "mêmes", "merci", "mes", "mien", "mienne", "miennes", "miens", "mille", "mince", "mine", "moi", "moi-même", "moins", "mon", "mot", "moyennant", "n", "na", "ne", "néanmoins", "neuf", "neuvième", "ni", "nombreuses", "nombreux", "nommés", "non", "nos", "notre", "nôtre", "nôtres", "nous", "nous-mêmes", "nouveaux", "nul", "o", "ô", "o|", "oh", "ohé", "olé", "ollé", "on", "ont", "onze", "onzième", "ore", "ou", "où", "ouf", "ouias", "oust", "ouste", "outre", "p", "paf", "pan", "par", "parce", "parmi", "parole", "partant", "particulier", "particulière", "particulièrement", "pas", "passé", "pendant", "personne", "personnes", "peu", "peut", "peuvent", "peux", "pff", "pfft", "pfut", "pièce", "pif", "plein", "plouf", "plupart", "plus", "plusieurs", "plutôt", "pouah", "pour", "pourquoi", "premier", "première", "premièrement", "près", "proche", "psitt", "puisque", "q", "qu", "quand", "quant", "quanta", "quant-à-soi", "quarante", "quatorze", "quatre", "quatre-vingt", "quatrième", "quatrièmement", "que", "quel", "quelconque", "quelle", "quelles", "quelque", "quelques", "quelqu'un", "quels", "qui", "quiconque", "quinze", "quoi", "quoique", "r", "revoici", "revoilà", "rien", "s", "sa", "sacrebleu", "sans", "sapristi", "sauf", "se", "seize", "selon", "sept", "septième", "sera", "seront", "ses", "seulement", "si", "sien", "sienne", "siennes", "siens", "sinon", "six", "sixième", "soi", "soi-même", "soit", "soixante", "son", "sont", "sous", "soyez", "stop", "suis", "suivant", "sujet", "sur", "surtout", "t", "ta", "tac", "tandis", "tant", "te", "té", "tel", "telle", "tellement", "telles", "tels", "tenant", "tes", "tic", "tien", "tienne", "tiennes", "tiens", "toc", "toi", "toi-même", "ton", "touchant", "toujours", "tous", "tout", "toute", "toutes", "treize", "trente", "très", "trois", "troisième", "troisièmement", "trop", "tsoin", "tsouin", "tu", "u", "un", "une", "unes", "uns", "v", "va", "vais", "valeur", "vas", "vé", "vers", "via", "vif", "vifs", "vingt", "vivat", "vive", "vives", "vlan", "voici", "voie", "voient", "voilà", "vont", "vos", "votre", "vôtre", "vôtres", "vous", "vous-mêmes", "vu", "w", "x", "y", "z", "zut","a", "abord", "absolument", "afin", "ah", "ai", "aie", "aient", "aies", "ailleurs", "ainsi", "ait", "allaient", "allo", "allons", "allô", "alors", "anterieur", "anterieure", "anterieures", "apres", "après", "as", "assez", "attendu", "au", "aucun", "aucune", "aucuns", "aujourd", "aujourd'hui", "aupres", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autant", "autre", "autrefois", "autrement", "autres", "autrui", "aux", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avoir", "avons", "ayant", "ayez", "ayons", "b", "bah", "bas", "basee", "bat", "beau", "beaucoup", "bien", "bigre", "bon", "boum", "bravo", "brrr", "c", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cependant", "certain", "certaine", "certaines", "certains", "certes", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chacun", "chacune", "chaque", "cher", "chers", "chez", "chiche", "chut", "chère", "chères", "ci", "cinq", "cinquantaine", "cinquante", "cinquantième", "cinquième", "clac", "clic", "combien", "comme", "comment", "comparable", "comparables", "compris", "concernant", "contre", "couic", "crac", "d", "da", "dans", "de", "debout", "dedans", "dehors", "deja", "delà", "depuis", "dernier", "derniere", "derriere", "derrière", "des", "desormais", "desquelles", "desquels", "dessous", "dessus", "deux", "deuxième", "deuxièmement", "devant", "devers", "devra", "devrait", "different", "differentes", "differents", "différent", "différente", "différentes", "différents", "dire", "directe", "directement", "dit", "dite", "dits", "divers", "diverse", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dixième", "doit", "doivent", "donc", "dont", "dos", "douze", "douzième", "dring", "droite", "du", "duquel", "durant", "dès", "début", "désormais", "e", "effet", "egale", "egalement", "egales", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "enfin", "entre", "envers", "environ", "es", "essai", "est", "et", "etant", "etc", "etre", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eux-mêmes", "exactement", "excepté", "extenso", "exterieur", "eûmes", "eût", "eûtes", "f", "fais", "faisaient", "faisant", "fait", "faites", "façon", "feront", "fi", "flac", "floc", "fois", "font", "force", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gens", "h", "ha", "haut", "hein", "hem", "hep", "hi", "ho", "holà", "hop", "hormis", "hors", "hou", "houp", "hue", "hui", "huit", "huitième", "hum", "hurrah", "hé", "hélas", "i", "ici", "il", "ils", "importe", "j", "je", "jusqu", "jusque", "juste", "k", "l", "la", "laisser", "laquelle", "las", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "longtemps", "lors", "lorsque", "lui", "lui-meme", "lui-même", "là", "lès", "m", "ma", "maint", "maintenant", "mais", "malgre", "malgré", "maximale", "me", "meme", "memes", "merci", "mes", "mien", "mienne", "miennes", "miens", "mille", "mince", "mine", "minimale", "moi", "moi-meme", "moi-même", "moindres", "moins", "mon", "mot", "moyennant", "multiple", "multiples", "même", "mêmes", "n", "na", "naturel", "naturelle", "naturelles", "ne", "neanmoins", "necessaire", "necessairement", "neuf", "neuvième", "ni", "nombreuses", "nombreux", "nommés", "non", "nos", "notamment", "notre", "nous", "nous-mêmes", "nouveau", "nouveaux", "nul", "néanmoins", "nôtre", "nôtres", "o", "oh", "ohé", "ollé", "olé", "on", "ont", "onze", "onzième", "ore", "ou", "ouf", "ouias", "oust", "ouste", "outre", "ouvert", "ouverte", "ouverts", "o|", "où", "p", "paf", "pan", "par", "parce", "parfois", "parle", "parlent", "parler", "parmi", "parole", "parseme", "partant", "particulier", "particulière", "particulièrement", "pas", "passé", "pendant", "pense", "permet", "personne", "personnes", "peu", "peut", "peuvent", "peux", "pff", "pfft", "pfut", "pif", "pire", "pièce", "plein", "plouf", "plupart", "plus", "plusieurs", "plutôt", "possessif", "possessifs", "possible", "possibles", "pouah", "pour", "pourquoi", "pourrais", "pourrait", "pouvait", "prealable", "precisement", "premier", "première", "premièrement", "pres", "probable", "probante", "procedant", "proche", "près", "psitt", "pu", "puis", "puisque", "pur", "pure", "q", "qu", "quand", "quant", "quant-à-soi", "quanta", "quarante", "quatorze", "quatre", "quatre-vingt", "quatrième", "quatrièmement", "que", "quel", "quelconque", "quelle", "quelles", "quelqu'un", "quelque", "quelques", "quels", "qui", "quiconque", "quinze", "quoi", "quoique", "r", "rare", "rarement", "rares", "relative", "relativement", "remarquable", "rend", "rendre", "restant", "reste", "restent", "restrictif", "retour", "revoici", "revoilà", "rien", "s", "sa", "sacrebleu", "sait", "sans", "sapristi", "sauf", "se", "sein", "seize", "selon", "semblable", "semblaient", "semble", "semblent", "sent", "sept", "septième", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "seul", "seule", "seulement", "si", "sien", "sienne", "siennes", "siens", "sinon", "six", "sixième", "soi", "soi-même", "soient", "sois", "soit", "soixante", "sommes", "son", "sont", "sous", "souvent", "soyez", "soyons", "specifique", "specifiques", "speculatif", "stop", "strictement", "subtiles", "suffisant", "suffisante", "suffit", "suis", "suit", "suivant", "suivante", "suivantes", "suivants", "suivre", "sujet", "superpose", "sur", "surtout", "t", "ta", "tac", "tandis", "tant", "tardive", "te", "tel", "telle", "tellement", "telles", "tels", "tenant", "tend", "tenir", "tente", "tes", "tic", "tien", "tienne", "tiennes", "tiens", "toc", "toi", "toi-même", "ton", "touchant", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "tres", "trois", "troisième", "troisièmement", "trop", "très", "tsoin", "tsouin", "tu", "té", "u", "un", "une", "unes", "uniformement", "unique", "uniques", "uns", "v", "va", "vais", "valeur", "vas", "vers", "via", "vif", "vifs", "vingt", "vivat", "vive", "vives", "vlan", "voici", "voie", "voient", "voilà", "voire", "vont", "vos", "votre", "vous", "vous-mêmes", "vu", "vé", "vôtre", "vôtres", "w", "x", "y", "z", "zut", "à", "â", "ça", "ès", "étaient", "étais", "était", "étant", "état", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"]

In [ ]:
# Document term Matrix
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=french_stop_words, ngram_range=(0,1))
data_cv = cv.fit_transform(corpus_fr['desc'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = corpus_fr['name']
data_dtm

In [ ]:
data_dtm = data_dtm.transpose()
sparse_counts = scipy.sparse.csr_matrix(data_dtm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v,k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [ ]:
# keep only nouns
from nltk import word_tokenize, pos_tag

def get_nouns(text):
    '''Keep Nouns only from Text'''
    is_noun = lambda pos: pos[:2] == "NN"
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [ ]:
corpus_fr['desc'] = corpus_fr['desc'].apply(lambda x : get_nouns(x))

In [ ]:
# Document term Matrix
from sklearn.feature_extraction.text import CountVectorizer

new_stop_words = ['poste', 'localisation', 'description', 'quelqu', 'gt', 'li',
                 'lt', 'france', 'expérience']
stop_words = french_stop_words + new_stop_words

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(corpus_fr['desc'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = corpus_fr['name']

data_dtm = data_dtm.transpose()
sparse_counts = scipy.sparse.csr_matrix(data_dtm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v,k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=10)
lda.print_topics()

In [ ]:
def get_nouns_adj(text):
    '''Keep Nouns only from Text'''
    is_noun = lambda pos: pos[:2] == "NN" or pos[:2] == "JJ" 
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [ ]:
corpus_fr['desc'] = corpus_fr['desc'].apply(lambda x : get_nouns_adj(x))

In [ ]:
# Document term Matrix
from sklearn.feature_extraction.text import CountVectorizer

new_stop_words = ['poste', 'localisation', 'description', 'quelqu', 'gt', 'li',
                 'lt', 'france', 'expérience']
stop_words = french_stop_words + new_stop_words

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(corpus_fr['desc'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = corpus_fr['name']

data_dtm = data_dtm.transpose()
sparse_counts = scipy.sparse.csr_matrix(data_dtm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v,k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10, minimum_probability=0)
lda.print_topics()

In [ ]:
#Topics for all job posts
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=15, minimum_probability=0)
lda.print_topics()

In [ ]:
lda_corpus = lda[corpus]

In [ ]:
[doc for doc in lda_corpus]

In [ ]:
from itertools import chain
scores = list(chain(*[[score for topic_id, score in topic] \
                    for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print(threshold)

In [ ]:
cluster1 = [j for i, j in zip(lda_corpus, df_fr.index) if i[0][1] > threshold]
cluster2 = [j for i, j in zip(lda_corpus, df_fr.index) if i[1][1] > threshold]
cluster3 = [j for i, j in zip(lda_corpus, df_fr.index) if i[2][1] > threshold]
#cluster4 = [j for i, j in zip(lda_corpus, df_fr.index) if i[3][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
#print(len(cluster4))

In [ ]:
df.iloc[cluster1]

In [ ]:
df_fr['topic'] = ''
df_fr.iloc[cluster1]['topic'] = lda.print_topic(0).split(" + ")[0].split('"')[1]
df_fr.iloc[cluster2]['topic'] = lda.print_topic(1).split(" + ")[0].split('"')[1]
df_fr.iloc[cluster3]['topic'] = lda.print_topic(2).split(" + ")[0].split('"')[1]

In [ ]:
lda.print_topics()

In [ ]:
df_fr.head()

In [ ]:
df['topic'].value_counts()